### EXERCISE 1.1.1. 

If unemployment compensation increases unemployment duration,
should we conclude that increasing such compensation is detrimental to society?
Provide some thoughts on this question in the context of the McCall model.

Answer:

Although it increases the unemployment duration, agents choose to be unemployed to maximize their utilities or EPV. Hence, from this perspective, it is not detrimental to the society.

In one extreme, if $c$ is very very high, it also possibly suggested this is a highly developed and automated world where people can have high unemployment compensation to enjoy instead of work.

### EXERCISE 1.1.2. 

The optimal action at time $t=0$ is determined by a time zero
reservation wage $w^*_0$ . The worker should accept the time zero wage offer if and only
if $w_0$ exceeds $w^*_0$ . Calculate $w^*_0$ for this problem, by analogy with $w^*_1$ in (1.4).


**Answer**

Let $h_0 = c+ \beta\sum_{w'\in W} v_1(w')\varphi(w')$ be the continuation value at time $t=0$.

Since the stopping value at $t=0$ is

$$
w_0 + \beta w_0 +\beta^2 w_0 = w_0(1+\beta +\beta^2)
$$

The reservation wage at $t=0$ is the wage equating the stopping value and the continuation value at time $t=0$, i.e.,

$$
w_0^*(1+\beta +\beta^2) = h_0
$$

This implies, 

$$
w_0^* = \frac{h_0}{1+\beta+\beta^2}
$$

### EXERCISE 1.1.3. 

Extend the above arguments to $T$ time periods, where $T$ can be
any finite number. Using Julia or another programming language, write a function
that takes $T$ as an argument and returns $(w^*_0,\ldots,w^*_T)$, the sequence of reservation
wages for each period.

First, we start with importing numpy for numerical operations and namedtuple to store the model parameters.

In [1]:
import numpy as np
from collections import namedtuple

A `namedtuple` is a convenient way to define a class. This type of data structure allows us to create tuple-like objects that have fields accessible by attribute lookup as well as being indexable and interable. 

In this model, we want to use the `namedtuple` to store values of the model, hence, we name it as `Model`. It requires the following parameters:

- `c`: the unemployment compensation
- `w_vals`: $W$, the finite wage space
- `n`: the cardinality of the wage space (in the following code, I use the uniform distribution, hence, this simplies the answer by not including extra parameters for the pdf)
- `β`: the discount factor

In [2]:
Model = namedtuple("Model", ("c", "w_vals", "n", "β","T"))

Then we use a function to input specific values into the `namedtuple`, i.e.,

In [3]:
def create_job_search_model(
    n = 50,          # wage grid size
    w_min = 11,      # lowest wage
    w_max = 60,      # highest wage
    c = 10,          # unemployment compensation
    β = 0.96,        # discount factor
    T = 10           # number of periods t= 0, 1,...,T
):
    """
    This function input the paramters with the above default values, and return a namedtuple
    """
    w_vals = np.linspace(w_min,w_max, n) # create a evenly spaced numbers over the specified intervals, with specified number of sample
    
    return Model(c = c, w_vals = w_vals, n = n, β = β, T=T) # return the namedtuple with the input parameters

Now we define a function that iteratively obtain the continuation value, and reservation wages

In [4]:
def reservation_wage(model):
    c, w_vals, n, β, T = model.c, model.w_vals, model.n, model.β, model.T  # Input the model parameters
    H = np.zeros(T+1)  # Initialize the continuation value sequence
    R = np.zeros(T+1)  # Initialize the reservation wage sequence
    S = np.zeros((T+1,n))  # Initialize the maximum values at each state at each period
    H[T] = c         # Input the last continuation value which is just the unemployment compensation
    R[T] = c         # The reservation wage at the last period is just the unemployment compensation
    S[T,:] = np.maximum(c, w_vals) # At period T, it is just comparing the unemployment compensation with the wages
    for t in range(1, T+1):
        H[T-t] = c + β * np.mean(S[T-t+1,:]) # Assuming uniform distribution, we only need to calculate the mean
        df = np.geomspace(1, β**t, t+1)   # this generate the sequence for the denominator
        dfs = np.sum(df)  # this is the denominator for the reservation wage calculation
        R[T-t] = H[T+1-t]/dfs    # This calculate the reservation wage at time T-t
        S[T-t,:] = np.maximum(dfs * w_vals, H[T-t])   # This returns the maximum values for each wage state by comparing the continuation value and stopping value
    return R
    

This function iteratively generate the reservation wage sequence. We can show the result by create the model and use this function to calculate the reservation wage sequence.

In [5]:
model = create_job_search_model()
reservation_wage(model)

array([36.50032766, 35.38365172, 34.07207926, 32.50704612, 30.6067209 ,
       28.23780776, 25.19318638, 21.10849802, 15.29705719,  5.10204082,
       10.        ])

### EXERCISE 1.2.1. 
Let $\alpha, s$ and $t$ be real numbers. Show that $\alpha \vee (s+t) \le s+\alpha \vee t$ whenever $s\ge 0$

**Proof**

Book-keeping:

$a\vee b$ means: $\max\{a,b\}$. Hence,

$$
\alpha \vee (s+t) \equiv \max\{\alpha, s+t\}
$$

and

$$
s + \alpha \vee t \equiv s+\max\{\alpha,t\}
$$

Suppose $\alpha \ge t$, this implies,

$$
s+\alpha \ge \alpha , s+\alpha \ge s+t
$$

Hence, the inequality holds.

Suppose $\alpha < t$, this implies, $\alpha < s+t$, hence the inequality also holds. This completes the proof.

### EXERCISE 1.2.2. 

Verify that the $\ell_1$ norm on $\mathbb{R}^n$ satisfies (a)–(d) above.

**Answer**

Book-keeping:

$\ell_1$ norm on $\mathbb{R}^n$ is defined as

$$
\|u\|_1 = \sum_{i=1}^n |u_i|
$$

Four conditions:

- [x] non-negativity: $\|u\|_1\ge 0$

$$
\|u\|_1 = \sum_{i=1}^n |u_i|\ge 0 
$$

- [x] Positive definiteness: $\|u\|_1 = 0 \iff u = 0$

$$
\|u\|_1 = 0 \iff u = 0
$$

- [x] Absolute homogeneity

$$
\|\alpha u\|_1 = \sum_{i=1}^n |\alpha u_i| = |\alpha|\sum_{i=1}^n |u_i| = \alpha \|u\|_1
$$

- [x] Triangle inequality

$$
\|u+v\|_1 = \sum_{i=1}^n |u_i+v_i| \le \sum_{i=1}^n|u_i|+|v_i| = \sum_{i=1}^n |u_i| + \sum_{i=1}^n|v_i| = \|u\|_1+\|v\|_1
$$


### EXERCISE 1.2.3. 

Fix $p\in \mathbb{R}^n$ with $p_i>0$ for all $i\in[n]$ and $\sum_{i}p_i=1$. 

Show that 

$$
\|u\|_{1,p} := \sum_{i=1}^n |u_i|p_i
$$

is a norm on $\mathbb{R}^n$.


**Answer**


Four conditions:

- [x] non-negativity: $\|u\|_{1,p}\ge 0$

$$
\|u\|_{1,p} = \sum_{i=1}^n |u_i|p_i\ge 0 
$$

- [x] Positive definiteness: $\|u\| = 0 \iff u = 0$

$$
\|u\|_{1,p} = 0 \iff u = 0
$$

- [x] Absolute homogeneity

$$
\|\alpha u\|_{1,p} = \sum_{i=1}^n |\alpha u_i|p_i = |\alpha|\sum_{i=1}^n |u_i|p_i = \alpha \|u\|_{1,p}
$$

- [x] Triangle inequality

$$
\|u+v\|_{1,p} = \sum_{i=1}^n |u_i+v_i|p_i \le \sum_{i=1}^n\left(|u_i|+|v_i|\right)p_i = \sum_{i=1}^n |u_i|p_i + \sum_{i=1}^n|v_i|p_i = \|u\|_{1,p}+\|v\|_{1,p}
$$

**Remark**

This implies expectation (absolute) is a norm

### EXERCISE 1.2.4

Show that the supremum norm is a norm on $\mathbb{R}^n$.

**Proof**

Book keeping:

Supremum norm of $u\in\mathbb{R}^n$:

$$
\|u\|_\infty = \max_{i}|u_i|
$$


Four conditions:

- [x] non-negativity: $\|u\|\ge 0$

$$
\|u\|_infty = \max|u_i|\ge 0  
$$

- [x] Positive definiteness: $\|u\| = 0 \iff u = 0$

$$
\|u\|_\infty = 0 \iff u = 0
$$

- [x] Absolute homogeneity

$$
\|\alpha u\|_\infty = \max|\alpha u_i| = |\alpha|\max|u_i|= |\alpha| \|u\|_{\infty}
$$

- [x] Triangle inequality

$$
\|u+v\|_\infty = \max|u_i+v_i| \le \max|u_i| + \max|u_i| = \|u\|_\infty + \|v\|_\infty
$$


### EXERCISE 1.2.5

Show that $\ell_0$ 'norm' 

$$
\|u\|_0 = \sum_{i=1}^n \mathbb{1}\{u_i\neq 0\}
$$

is not a norm.

**Proof**

It violates the absolute homogeneity condition. 

$$
\|\alpha u\|_0 = \sum_{i=1}^n \mathbb{1}\{\alpha u_i \neq 0\} =  \sum_{i=1}^n \mathbb{1}\{ u_i \neq 0\} = \|u\|_0\neq |\alpha|\|u\|_0
$$



### EXERCISE 1.2.6

Let's write $\|\cdot\|_a\sim \|\cdot\|_b$ if there exist finite $M,N$ such that

$$
M\|u\|_a\le \|u\|_b \le N\|u\|_a \,\,\forall u\in\mathbb{R}^n
$$

holds. Prove that $\sim$ is an equivalence relation on the set of all norms on $\mathbb{R}^n$.


**Proof**

Book Keeping:

Conditions of Equivalence relation:

- [x] Reflectivity: $\|\cdot \|_a \sim \|\cdot \|_a$

Take $M=N=1$. 

- [x] Symmetry: $\|\cdot\|_a \sim \|\cdot\|_b \implies \|\cdot\|_b\sim \|\cdot\|_a$

Suppose $\|\cdot\|_a \sim \|\cdot\|_b$, we have,

$$
M\|u\|_a\le \|u\|_b \le N\|u\|_a \,\,\forall u\in\mathbb{R}^n
$$

This implies 

$$
\frac{1}{N}\|u\|_b \le \|u\|_a\le \frac{1}{M} \|u\|_b \,\,\forall u\in\mathbb{R}^n
$$

where, $\frac{1}{N},\frac{1}{M}$ are finite (assume $M,N\neq 0$). Hence, $\|\cdot\|_b\sim \|\cdot \|_a$.


- [x] Transitivity: $\|\cdot\|_a\sim \|\cdot\|_b$, $\|\cdot\|_b\sim\|\cdot\|_c \implies \|\cdot\|_a\sim\|\cdot\|_c$

Suppose $\|\cdot\|_a\sim \|\cdot\|_b$ and $\|\cdot\|_b\sim\|\cdot\|_c$, then we have,

$$
M\|u\|_a\le \|u\|_b \le N\|u\|_a \,\,\forall u\in\mathbb{R}^n
$$

$$
M'\|u\|_b\le \|u\|_c \le N'\|u\|_b \,\,\forall u\in\mathbb{R}^n
$$


This implies 

$$
MM'\|u\|_a\le M'\|u\|_b \le \|u\|_c\le N'\|u\|_b \le NN'\|u\|_a \,\,\forall u\in\mathbb{R}^n
$$

Since $MM',NN'$ are also finite, hence, the transitivity is satisfied.

### EXERCISE 1.2.7 

Let $\|\cdot\|_a$ and $\|\cdot\|_b$ be any two norms on $\mathbb{R}^n$. Given a point $u$ in $\mathbb{R}^n$ and a sequence $(u_m)$ in $\mathbb{R}^n$. Use

$$
M\|u\|_a\le \|u\|_b \le N\|u\|_a \,\,\forall u\in\mathbb{R}^n
$$

to confirm that $\|u_m-u\|_a\to 0\implies \|u_m-u\|\to 0$ as $m\to \infty$.


**Proof**

Suppose $\|u_m-u\|_a\to 0$ as $m\to\infty$. Hence, for all $\epsilon>0$, there exists $M(\epsilon)\in \mathbb{N}$ such that, $m\ge M$ implies 

$$
\|u_m-u\|_a <\frac{\epsilon}{N}
$$

this implies, by the given equivalence, we have

$$
\|u_m-u\|_b\le N\|u_m-u\|_a<\epsilon
$$

Hence, we have $\|u_m-u\|_b\to 0$ as $m\to0$.

### EXERCISE 1.2.8 

Let $\|\cdot\|$ be any norm on $\mathbb{R}^n$. Fixing a point $u\in\mathbb{R}^n$ and a sequence $(u_m)$ in $\mathbb{R}^n$.

Let $u^i$ and $u^i_m$ be the $i-th$ component of $u$ and $u_m$ respectively. 

Show that $u_m^i\to u^i$ for all $i\in \{1,2,\ldots,n\}$ if and only if $\|u_m-u\|\to 0$.

**Proof**

$(\implies)$

Suppose $u_m^i \to u^i$ for all $i\in\{1,2,\ldots,n\}$. This implies for all $\epsilon>0$, there exist $M^i(\epsilon)\in\mathbb{N}$ such that, $m^i\ge M^i$ implies 

$$
|u_m^i -u^i|<\frac{\epsilon}{n}
$$

for all $i=1,2,\ldots,n$.

Take $M = \max\{M^1,M^2,\ldots,M^n\}$. $M$ exists as $n$ is finite. Then, we have for all $\epsilon>0$, there exist $M$ as defined above, such that $m^i\ge M$, we have

$$
\|u_m-u\| = \sum_{i=1}^n |u_m^i-u^i|< n\frac{\epsilon}{n} = \epsilon
$$

Hence, $\|u_m-u\|\to 0$ as $m\to 0$.


$(\impliedby)$

Suppose $\|u_m-u\|\to 0$ as $m\to 0$. This means for all $\epsilon>0$, there exists $M(\epsilon)>0$ such that $m\ge M$ implies 

$$
|u_m^i-u^i|\le \|u_m-u\| = \sum_{i}^n |u_m^i-u^i|<\epsilon
$$

for all $i=1,2,\ldots, n$.

This completes the proof.


### EXERCISE 1.2.9 

Show that the operator $\|\cdot \|_o$ is submultiplicative on $\mathbb{R}^{n\times n}$.

Provide a counterexample to claim that $\|\cdot\|_\infty$ is submultiplicative.


**Proof**

Book Keeping:

**Submultiplicative**:

For all $A, B \in\mathbb{R}^{n\times n}$,

$$
\|AB\|\le \|A\|\|B\|
$$

Definition of operator $\|\cdot\|_o$:

$$
\|B\|_o:= \max_{\|u\|=1}\|Bu\|
$$

where $B\in\mathbb{R}^{n\times n}, u\in\mathbb{R}^n$

From the definition, we get,

$$
\|AB\|_o = \max_{\|u\|=1}\|ABu\|
$$

Let $C:= AB = [C_ij]$, where $C_ij$ is the $i,j$th element of $C$. Since $u$ is on the unit circle, 